<a href="https://colab.research.google.com/github/Rogerio-mack/work/blob/main/Python_dir_files_reg_html_to_gift.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Processa arquivos de questões html no diretório /content e gera em /content/output arquivo de questões gift

In [18]:
import re
# as per recommendation from @freylis, compile once only
# CLEANR = re.compile('<.*?>') 
CLEANR = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')

def cleanhtml(raw_html):
  cleantext = re.sub(CLEANR, '', raw_html)
  return cleantext

  

In [19]:
def process(path, f, outputpath):
  infile = open(path + '/' + f,'r') 
  outfile = open(outputpath + '/' + f.replace('q','g'),'w')

  roman = ['\\ni. ', '\\nii. ', '\\niii. ', '\\niv. ', '\\nv. ' ]
  oli = False
  ioli = 0

  ola = False
  iola = 0

  resp = ''
  resps = {'a':0,'b':1,'c':2,'d':3,'e':4}

  q = f[1:5].split('.')[0]
  
  for line in infile:

    if resp == '':                                      # Resposta
      resp = resps[ line[2] ]
      continue

    if line[0:4] == '<img':                            # Se inserida imagem na questão
      texto = line
      outfile.write(texto)
      continue

    line = line.replace('Questão #q#','(Questão_' + q + ')\\n')  

    if line.count('<ol type="i">') != 0:                # Trata lista de frases
      line = line.replace('<ol type="i">','\\n',1)
      oli = True

    if oli:  
      if line.count('<li>') != 0:
        line = line.replace('<li>',roman[ioli])
        ioli = ioli + 1
      line = line.replace('</li>','')
      if line.count('</ol>') != 0:
        line = line.replace('</ol>','\\n')
        oli = False
        ioli = 0
      line = line.strip(' ')  

    if line.count('<ol type="a">') != 0:               # Trata lista de alternativas
      line = line.replace('<ol type="a">','{',1)
      ola = True

    line = line.replace(':','\:')                      # Caracteres especiais
    line = line.replace('=','\=') 

    if ola:  
      if line.count('<li>') != 0:
        if iola == resp:
          line = line.replace('<li>','= ')
        else:
          line = line.replace('<li>','~ ') 
        iola = iola + 1
      line = line.replace('</li>','')
      if line.count('</ol>') != 0:
        line = line.replace('</ol>','}')
        ola = False
        iola = 0
      line = line.strip(' ')  

    texto = cleanhtml(line)

    texto = texto.replace('\:','\:\\n')                # Salta linhas
    outfile.write(texto)

  infile.close()
  outfile.close()

  infile = open(outputpath + '/' + f.replace('q','g'),'r')
  print(infile.read())
  infile.close()

  return


In [ ]:
import os
path = '/content'
files = os.listdir(path)

if not(os.path.exists('/content/output')):
  os.mkdir('/content/output')

for f in files:
  print(10*'* ' + f + ' ' + 10*'* ')
  if f[0] == 'q':
    process('/content', f, '/content/output')

In [ ]:
import os
path = '/content/output'

if os.path.exists(path + '/' + 'final_gift.txt'):
  os.remove(path + '/' + 'final_gift.txt')

files = os.listdir(path)

for f in sorted(files):
  if not(os.path.exists(path + '/' + 'final_gift.txt')):
    outfile = open(path + '/' + 'final_gift.txt','w')
  
  print(10*'* ' + f + ' ' + 10*'* ')
  infile = open(path + '/' + f,'r') 
  for line in infile:
    outfile.write(line)
  infile.close()
  
outfile.close()

infile = open(path + '/' + 'final_gift.txt','r')
print(infile.read())
infile.close()